In [1]:
import numpy as np

In [2]:
def create_input_data_file(samples_count, min_value, max_value, file_name="test_input.txt", rand_state=1):
    """ Create text file with samples of input data for testbench.
    
        Every string has format: <data> <delta> <min_value> <max_value>, where <data> - integer in [min_value, max_value];
            delta - integer in [- M // 2, M // 2) for even M or [- (M - 1) // 2, (M - 1) // 2] for odd M, 
            M = max_value - min_value + 1; min_value, max_value - integers 0 < min_value < max_value
        Arguments:
        - samples_count - number of strings in file, integer > 0;
        - min_value - low boundary for appropriate data sample, i.e. min_value[ii] <= data[ii], 
            array [1, samples_count] or integer - one boundary for all data samples;
        - max_value - high boundary for appropriate data sample, i.e. data[ii] <= max_value[ii], 
            array [1, samples_count] or integer - one boundary for all data samples;            
        - file_name - string with name of created file
        - rand_state - state of random generator, for samples repeatability
        
        return array [samples_count, 4] of integers where each row has an appropriate string content from file.
    """
    assert type(min_value) in [int, np.ndarray], "Error! Min_value must be integer or array."
    assert type(max_value) == type(min_value), "Error! Max_value and min_value must be the same type."
    assert isinstance(min_value, int) or (min_value.size == len(min_value) == samples_count), "Error! Min_value must be " +\
        f"integer or array. If min_value is array then it must be vector with length = {samples_count}, " + \
        f"but has shape {min_value.shape}."
    assert isinstance(max_value, int) or (max_value.shape == min_value.shape), "Error! Max_value must be " +\
        f"integer or array. If max_value is array then it must be vector with length = {samples_count}, " + \
        f"but has shape {max_value.shape}."  

    
    # generate array with samples
    rgen = np.random.RandomState(seed=rand_state)
    modulo = max_value - min_value + 1
    if isinstance(min_value, np.ndarray):
        pass
        min_value = min_value.reshape(-1, 1)
        max_value = max_value.reshape(-1, 1)
        modulo = modulo.reshape(-1, 1)        
        data, delta = (np.zeros_like(modulo) for _ in range(2))
        for ii, (cur_min, cur_max, cur_mod) in enumerate(zip(min_value, max_value, modulo)):
            data[ii, 0] = rgen.randint(cur_min, cur_max + 1)
            delta[ii, 0] = rgen.randint(- cur_mod // 2, cur_mod //2) if cur_mod % 2 == 0 else \
                rgen.randint(- (cur_mod - 1) // 2, (cur_mod - 1) // 2 + 1)
        mod_data = np.hstack((data, delta, min_value, max_value)).astype(int)
    elif isinstance(min_value, int):
        data = rgen.randint(min_value, max_value + 1, size=(samples_count, 1))
        delta = rgen.randint(- modulo // 2, modulo // 2, size=(samples_count, 1)) if modulo % 2 == 0 else \
            rgen.randint(- (modulo - 1) // 2, (modulo - 1) // 2 + 1, size=(samples_count, 1))
        min_max_vec = np.array([min_value, max_value]) * np.ones((samples_count, 1))
        mod_data = np.hstack((data, delta, min_max_vec)).astype(int)
        
    # write sample to file        
    with open(file_name, 'w') as f:
        strings = []
        for ii in range(samples_count):
            strings.append(f"{mod_data[ii, 0]:d} {mod_data[ii, 1]:d} {mod_data[ii, 2]} {mod_data[ii, 3]}\n")
        f.writelines(strings)
    return mod_data

def create_output_data_file(input_data_file="test_input.txt", output_data_file="test_output.txt"):
    """ Create text file with samples of output data for testbench.
    
        Every string has format: <out_data>, where out_data - integer >= 0
        Arguments:
        - input_data_file - name (destination) of file with input data, string. Before execute function 
            you must create this file (using create_input_data_file(), for example) first.
        - output_data_file - name (destination of created file with output data). File contains
            <data> + <delta> wrapped to [min_value, max_value] from input_data_file in each string, 
            see create_input_data_file() help.
    
        return array [samples_count, 1] with sums written to file output_data_file.
    """
    try:
        with open(input_data_file, 'r') as f_rd:
            with open(output_data_file, 'w') as f_wr:
                out_data = None
                for input_string in f_rd.readlines():
                    data, delta, min_value, max_value = (int(el) for el in input_string.split(" "))
                    output_string = data + delta
                    if min_value <= output_string <= max_value:
                        pass
                    elif output_string > max_value:
                        output_string = (output_string - max_value) + min_value - 1
                    elif output_string < min_value:
                        output_string = max_value - (min_value - output_string) + 1
                    if out_data is not None:
                        out_data = np.vstack((out_data, output_string))
                    else:
                        out_data = np.copy(output_string)
                f_wr.writelines((f"{int(el)}\n" for el in out_data))
        return out_data
    except IOError:
        print(f"Error! File {input_data_file} is not accessible.")

In [3]:
show_files = False
check_files = True
samples_count = 20000
min_value = np.array([10, 2, 850, 0])
max_value = np.array([100, 10, 4000, 2**16 - 1])
min_val_vec, max_val_vec = None, None
for cur_min, cur_max in zip(min_value, max_value):
    new_mins = cur_min * np.ones(samples_count // len(min_value))
    new_maxs = cur_max * np.ones(samples_count // len(max_value))
    if min_val_vec is not None:
        min_val_vec = np.hstack((min_val_vec, new_mins))
        max_val_vec = np.hstack((max_val_vec, new_maxs))
    else:
        min_val_vec, max_val_vec = np.copy(new_mins), np.copy(new_maxs)
        
in_data = create_input_data_file(samples_count=samples_count, min_value=min_val_vec, max_value=max_val_vec, 
                                 file_name="test_input.txt", rand_state=100)

if show_files:
    with open("test_input.txt", 'r') as f:
        for data_str in f.readlines():
            print(data_str)

In [4]:
out_data = create_output_data_file(input_data_file="test_input.txt", output_data_file="test_output.txt")

if show_files:
    with open("test_output.txt", 'r') as f:
        for data_str in f.readlines():
            print(data_str)

In [5]:
if check_files:
    # get result using algorithm from vhdl-module
    M = in_data[:, 3] - in_data[:, 2] + 1
    in_to_mod = in_data[:, 0] - in_data[:, 2]
    delta_mod = np.mod(in_data[:, 1], M)
    sum_mod = np.mod(in_to_mod + delta_mod, M)
    sum_back_to_min_max = sum_mod + in_data[:, 2]
    assert np.all(sum_back_to_min_max.reshape(out_data.shape) == out_data), \
        "Error! Data from output_file don't match data from input_file."

In [6]:
print(out_data[:10])
print(out_data[-10:])

[[88]
 [28]
 [92]
 [27]
 [84]
 [13]
 [95]
 [83]
 [81]
 [53]]
[[30331]
 [55984]
 [ 9291]
 [  923]
 [ 7254]
 [12572]
 [21445]
 [ 7255]
 [12765]
 [12546]]
